In [1]:
import pandas as pd

In [2]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [10]:
neg = embed_bert_cls('Худший день в моей жизни. Пришёл на лабу, в чём смысл моей жизни?', model, tokenizer)
poz = embed_bert_cls('Ура! ЦСКА победил. Я так этому рад!', model, tokenizer)
poz2 = embed_bert_cls('Сегодня был отличный день. Много всего успел. Чувствую себя хорошо', model, tokenizer)
rev = embed_bert_cls('Нырять очень комфортно, очень понравилось. Правда вода была холодная', model, tokenizer)
cat1 = embed_bert_cls('дайвинг', model, tokenizer)
cat2 = embed_bert_cls('рестораны', model, tokenizer)
cat3 = embed_bert_cls('баскетбол', model, tokenizer)

In [11]:
def cos(a,b):
    return a@b/((a**2).sum() * (b**2).sum())

cos(poz, poz2)
cos(neg,poz2)

0.6510258

In [12]:
cos(rev, cat1), cos(rev, cat2), cos(rev, cat3)

(0.53169346, 0.42526814, 0.37361318)

In [14]:
df = pd.read_csv(r"d:\BMSTU_im_Baumana\семестры\Семестра_5\nlp\merged_data (1).csv")
df = df.drop("Unnamed: 0", axis=1)
df

,adress,name_ru,rating,rubrics,cleaned_text_without_stopwords
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,московский квартал шумно летом ночам дикие гон...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...
...,...,...,...,...,...
499995,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4,Железнодорожная станция,охрана кривая добрая двери закрываются автомат...
499996,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4,Железнодорожная станция,сравнению многими современными платформами эпо...
499997,"Новосибирск, Коммунистическая улица, 48А",NaN,5,"Бар, паб",приятная атмосфера прекрасное вино волшебная ж...
499998,"Астраханская область, Харабалинский район",Сарай-Бату,5,Достопримечательность,семьёй отличное место рекомендую это настоящий...


In [15]:
rubrics_unique = df.rubrics.unique()
rubrics_unique

array(['Жилой комплекс',
       'Магазин продуктов;Продукты глубокой заморозки;Магазин мяса, колбас',
       'Фитнес-клуб', ...,
       'Веломагазин;Ремонт велосипедов;Оптовая компания',
       'Шиномонтаж;Шины и диски;Автосвет',
       'Школа танцев;Студия йоги;Стретчинг'], dtype=object)

In [16]:
print(f"Количество рубрик в исходном dataset: {len(rubrics_unique)}")

Количество рубрик в исходном dataset: 31793


In [17]:
df = df.drop(['adress', 'name_ru', 'rating'], axis=1)
df

,rubrics,cleaned_text_without_stopwords
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...
...,...,...
499995,Железнодорожная станция,охрана кривая добрая двери закрываются автомат...
499996,Железнодорожная станция,сравнению многими современными платформами эпо...
499997,"Бар, паб",приятная атмосфера прекрасное вино волшебная ж...
499998,Достопримечательность,семьёй отличное место рекомендую это настоящий...


In [18]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [19]:
import openpyxl
data_rubrics = pd.read_excel('excele_analytic/privitive_rubrics_frequency.xlsx')
data_rubrics

,split_rubrics,count
0,Кафе,58496
1,Ресторан,56761
2,Гостиница,43133
3,Магазин продуктов,21346
4,Супермаркет,19746
...,...,...
1365,Фармацевтическое оборудование,1
1366,Центр дневного пребывания для пожилых,1
1367,Чистка и ремонт колодцев,1
1368,Экстренная социальная психологическая помощь,1


In [20]:
list_of_rubrics = data_rubrics.split_rubrics
list_of_rubrics = list_of_rubrics.tolist()
type(list_of_rubrics)

list

In [21]:
rubrics_vectors = []

def rubric_to_vector(list_of_rubrics):
    for i in range(len(list_of_rubrics)):
        rubrics_vectors.append(embed_bert_cls(list_of_rubrics[i], model=model, tokenizer=tokenizer))

In [22]:
rubric_to_vector(list_of_rubrics)

print(f"Размерность вектора: {len(rubrics_vectors)}")

Размерность вектора: 1370


In [23]:
data_rubrics["vector"] = rubrics_vectors
data_rubrics

,split_rubrics,count,vector
0,Кафе,58496,"[0.008358064, -0.0052378257, -0.009260751, -0...."
1,Ресторан,56761,"[-0.018206751, -0.042570315, -0.021068841, -0...."
2,Гостиница,43133,"[-0.026418814, -0.01931552, -0.043476027, -0.0..."
3,Магазин продуктов,21346,"[0.0021013357, -0.028616607, 0.01592993, -0.09..."
4,Супермаркет,19746,"[0.026823604, 0.026125867, -0.008277477, -0.08..."
...,...,...,...
1365,Фармацевтическое оборудование,1,"[0.019576546, -0.013938191, 0.059861995, -0.09..."
1366,Центр дневного пребывания для пожилых,1,"[0.039014414, 0.016539128, 0.066350244, -0.070..."
1367,Чистка и ремонт колодцев,1,"[-0.005404027, 0.05197321, 0.046515744, -0.062..."
1368,Экстренная социальная психологическая помощь,1,"[0.0028114265, -0.035814375, 0.105799705, -0.1..."


In [24]:
data_rubrics = data_rubrics.drop("count", axis=1)
data_rubrics

,split_rubrics,vector
0,Кафе,"[0.008358064, -0.0052378257, -0.009260751, -0...."
1,Ресторан,"[-0.018206751, -0.042570315, -0.021068841, -0...."
2,Гостиница,"[-0.026418814, -0.01931552, -0.043476027, -0.0..."
3,Магазин продуктов,"[0.0021013357, -0.028616607, 0.01592993, -0.09..."
4,Супермаркет,"[0.026823604, 0.026125867, -0.008277477, -0.08..."
...,...,...
1365,Фармацевтическое оборудование,"[0.019576546, -0.013938191, 0.059861995, -0.09..."
1366,Центр дневного пребывания для пожилых,"[0.039014414, 0.016539128, 0.066350244, -0.070..."
1367,Чистка и ремонт колодцев,"[-0.005404027, 0.05197321, 0.046515744, -0.062..."
1368,Экстренная социальная психологическая помощь,"[0.0028114265, -0.035814375, 0.105799705, -0.1..."


In [25]:
data_rubrics.to_csv("rubric_to_vector_windows.csv")

In [81]:
df

,rubrics,cleaned_text_without_stopwords
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...
...,...,...
499995,Железнодорожная станция,охрана кривая добрая двери закрываются автомат...
499996,Железнодорожная станция,сравнению многими современными платформами эпо...
499997,"Бар, паб",приятная атмосфера прекрасное вино волшебная ж...
499998,Достопримечательность,семьёй отличное место рекомендую это настоящий...


In [27]:
data_rubrics.iloc[0].vector.shape

(312,)

In [28]:
def cos_distance(a, b):
    return a@b/((a**2).sum() * (b**2).sum())

In [29]:
list_of_comments = df.cleaned_text_without_stopwords.tolist()
print(f"Длина списка комментариев: {len(list_of_comments)}")

Длина списка комментариев: 500000


In [30]:
comments_vectors = []

def rubric_to_vector(list_of_comments):
    for i in range(len(list_of_comments)):
        comments_vectors.append(embed_bert_cls(list_of_comments[i], model=model, tokenizer=tokenizer))

In [31]:
rubric_to_vector(list_of_comments)

len(comments_vectors)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [33]:
len(comments_vectors)

38187

In [36]:
%pip install Pympler


     ---------------------------------------- 0.0/164.8 kB ? eta -:--:--
     -------------- ------------------------ 61.4/164.8 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  163.8/164.8 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 164.8/164.8 kB 2.0 MB/s eta 0:00:00


In [38]:
import sys
from pympler.asizeof import asizeof

print(f"Занимает памяти: {asizeof(list_of_comments)} в байтах")
# maximum = 536,870,912

Занимает памяти: 300916456 в байтах


In [35]:
list_of_comments[38187] # ошибку выдаёт на nan

nan

In [39]:
df1 = df[:38187]

In [40]:
df1

,rubrics,cleaned_text_without_stopwords
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...
...,...,...
38182,Гостиница,целом норманая гостиница праздники единственно...
38183,Гостиница,единственный плюс чистый номер очень сильно па...
38184,АЗС,милоприятноудивительно красивое месторекомендую
38185,Магазин мебели;Мебельная фабрика;Мебель на заказ,заказывали данной компании шкаф кантри венге о...


In [41]:
df1["vector_text"] = comments_vectors
df1

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\469559564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["vector_text"] = comments_vectors


,rubrics,cleaned_text_without_stopwords,vector_text
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...,"[0.047512185, -0.021974338, 0.005056214, -0.05..."
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...,"[0.03578344, 0.045210324, -0.0045692245, -0.12..."
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...,"[0.024027942, -0.039055817, -0.06833198, -0.05..."
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...,"[0.110127084, -0.055666976, 0.024924437, -0.12..."
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...,"[0.050955813, 0.0081215715, 0.040434185, -0.05..."
...,...,...,...
38182,Гостиница,целом норманая гостиница праздники единственно...,"[0.028504416, 0.008679567, 0.012570469, -0.029..."
38183,Гостиница,единственный плюс чистый номер очень сильно па...,"[0.030039448, -0.019010512, -0.052233763, -0.0..."
38184,АЗС,милоприятноудивительно красивое месторекомендую,"[0.036341768, -0.00063123606, 0.032243636, -0...."
38185,Магазин мебели;Мебельная фабрика;Мебель на заказ,заказывали данной компании шкаф кантри венге о...,"[0.031307455, 0.018903187, -0.056326706, -0.09..."


In [46]:
accuracy = []

def accuracy_of_bert(df2, df3, accuracy, dr1, dr2):
    lenght = len(df2)
    lenght_rub = len(dr1)

    for i in range(lenght):
        predict = -2
        predict_rub = "NONE"
        for j in range(lenght_rub):
            cos = cos_distance(df3[i], dr2[j])
            if cos > predict:
                predict_rub = dr1[j]
                predict = cos
        
        accuracy.append(predict_rub)

In [47]:
df2 = df1.rubrics.tolist()
df3 = df1.vector_text.tolist()

dr1 = data_rubrics.split_rubrics.tolist()
dr2 = data_rubrics.vector.tolist()

accuracy_of_bert(df2, df3, accuracy, dr1, dr2)

print(f"Количество сделанных предиктов: {len(accuracy)}")

Количество сделанных предиктов: 38187


In [48]:
df1['predicted'] = accuracy
df1

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\3417906267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['predicted'] = accuracy


,rubrics,cleaned_text_without_stopwords,vector_text,predicted
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...,"[0.047512185, -0.021974338, 0.005056214, -0.05...",Станция скоростного городского транспорта
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...,"[0.03578344, 0.045210324, -0.0045692245, -0.12...",Магазин фиксированной цены
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...,"[0.024027942, -0.039055817, -0.06833198, -0.05...","Бюро пропусков, пост охраны"
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...,"[0.110127084, -0.055666976, 0.024924437, -0.12...",Одежда больших размеров
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...,"[0.050955813, 0.0081215715, 0.040434185, -0.05...",Салон бровей и ресниц
...,...,...,...,...
38182,Гостиница,целом норманая гостиница праздники единственно...,"[0.028504416, 0.008679567, 0.012570469, -0.029...",Салон вечерней одежды
38183,Гостиница,единственный плюс чистый номер очень сильно па...,"[0.030039448, -0.019010512, -0.052233763, -0.0...",Доставка еды и обедов
38184,АЗС,милоприятноудивительно красивое месторекомендую,"[0.036341768, -0.00063123606, 0.032243636, -0....",Достопримечательность
38185,Магазин мебели;Мебельная фабрика;Мебель на заказ,заказывали данной компании шкаф кантри венге о...,"[0.031307455, 0.018903187, -0.056326706, -0.09...",Фасовочно-упаковочное оборудование


In [52]:
l = df1.shape[0]

accuracy = []

for i in range(l):
    row = df1.iloc[i]
    accuracy.append(1 if row.predicted in row.rubrics else 0)

len(accuracy)

38187

In [53]:
df1['acc'] = accuracy
df1

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\2679048792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['acc'] = accuracy


,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...,"[0.047512185, -0.021974338, 0.005056214, -0.05...",Станция скоростного городского транспорта,0
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...,"[0.03578344, 0.045210324, -0.0045692245, -0.12...",Магазин фиксированной цены,0
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...,"[0.024027942, -0.039055817, -0.06833198, -0.05...","Бюро пропусков, пост охраны",0
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...,"[0.110127084, -0.055666976, 0.024924437, -0.12...",Одежда больших размеров,0
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...,"[0.050955813, 0.0081215715, 0.040434185, -0.05...",Салон бровей и ресниц,1
...,...,...,...,...,...
38182,Гостиница,целом норманая гостиница праздники единственно...,"[0.028504416, 0.008679567, 0.012570469, -0.029...",Салон вечерней одежды,0
38183,Гостиница,единственный плюс чистый номер очень сильно па...,"[0.030039448, -0.019010512, -0.052233763, -0.0...",Доставка еды и обедов,0
38184,АЗС,милоприятноудивительно красивое месторекомендую,"[0.036341768, -0.00063123606, 0.032243636, -0....",Достопримечательность,0
38185,Магазин мебели;Мебельная фабрика;Мебель на заказ,заказывали данной компании шкаф кантри венге о...,"[0.031307455, 0.018903187, -0.056326706, -0.09...",Фасовочно-упаковочное оборудование,0


In [55]:
df1[df1.acc==1]

,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...,"[0.050955813, 0.0081215715, 0.040434185, -0.05...",Салон бровей и ресниц,1
49,Фотоуслуги;Художественный салон;Магазин подарк...,покупал сувенирную продукцию симпатичное хорош...,"[0.0902368, -0.0036669855, -0.0293246, -0.0759...",Магазин подарков и сувениров,1
70,Магазин рыбы и морепродуктов,доступные цены широкий выбор свежая рыба часто...,"[0.033990063, -0.007741724, -0.006674127, -0.0...",Магазин рыбы и морепродуктов,1
96,Стоматологическая клиника,почитав отзывы решила пойти стоматологию сосед...,"[-0.00026073135, 0.022088248, 0.05264264, -0.0...",Стоматологическая клиника,1
103,Быстрое питание;Кафе,отличное место готовят вкусно быстро особое вн...,"[0.07191118, -0.0006154347, 0.01180784, -0.101...",Быстрое питание,1
...,...,...,...,...,...
38134,Суши-бар;Доставка еды и обедов;Магазин суши и ...,всем приветбрал несколько заказов суши супы це...,"[0.027236791, 0.0072074668, -0.0065498063, -0....",Магазин суши и азиатских продуктов,1
38145,Быстрое питание,вожу детей редко фостфуд моё быстро вежливоно ...,"[0.10134589, -0.013962053, 0.0014707376, -0.13...",Быстрое питание,1
38160,Товары для кондитеров;Магазин кулинарии,плохой магазин кондитеров тортоделов ассортиме...,"[0.08520487, 0.016659563, -0.008038059, -0.098...",Товары для кондитеров,1
38164,Магазин цветов;Товары для интерьера;Доставка ц...,очень милый цветочный салон девочки составили ...,"[0.1197234, 0.002828814, -0.05412488, -0.07911...",Доставка цветов и букетов,1


In [56]:
df1[df1.acc==0]

,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...,"[0.047512185, -0.021974338, 0.005056214, -0.05...",Станция скоростного городского транспорта,0
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...,"[0.03578344, 0.045210324, -0.0045692245, -0.12...",Магазин фиксированной цены,0
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...,"[0.024027942, -0.039055817, -0.06833198, -0.05...","Бюро пропусков, пост охраны",0
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...,"[0.110127084, -0.055666976, 0.024924437, -0.12...",Одежда больших размеров,0
5,Оператор сотовой связи;Интернет-провайдер,приятное общение доступно объяснили понравилось,"[0.01317609, -0.008376404, 0.031744435, -0.042...",Служба знакомств,0
...,...,...,...,...,...
38182,Гостиница,целом норманая гостиница праздники единственно...,"[0.028504416, 0.008679567, 0.012570469, -0.029...",Салон вечерней одежды,0
38183,Гостиница,единственный плюс чистый номер очень сильно па...,"[0.030039448, -0.019010512, -0.052233763, -0.0...",Доставка еды и обедов,0
38184,АЗС,милоприятноудивительно красивое месторекомендую,"[0.036341768, -0.00063123606, 0.032243636, -0....",Достопримечательность,0
38185,Магазин мебели;Мебельная фабрика;Мебель на заказ,заказывали данной компании шкаф кантри венге о...,"[0.031307455, 0.018903187, -0.056326706, -0.09...",Фасовочно-упаковочное оборудование,0


Что...

In [76]:
import numpy as np 

df.cleaned_text_without_stopwords.fillna(value=" ", inplace=True)

df2 = df[38178:60000]

In [77]:
list_of_comments_v2 = df2.cleaned_text_without_stopwords.tolist()
print(f"Длина списка комментариев 2: {len(list_of_comments_v2)}")

comments_vectors_part2 = ["ERROR"]

def rubric_to_vector(list_of_comments):
    for i in range(len(list_of_comments)):
        if list_of_comments[i] != " ":
            comments_vectors_part2.append(embed_bert_cls(list_of_comments[i], model=model, tokenizer=tokenizer))
        else:
            comments_vectors_part2.append("ERROR")

rubric_to_vector(list_of_comments_v2)

print(f"Размерность вектора: {len(comments_vectors_part2)}")

Длина списка комментариев 2: 21822
Размерность вектора: 21823


In [88]:
df2 = df[38177:60000]

In [89]:
df2

,rubrics,cleaned_text_without_stopwords
38177,Товары для праздника;Магазин подарков и сувениров,требуется организации праздника цены удовлетво...
38178,Фанера,давно работаю компанией фанерапитер отличное к...
38179,Фанера,заказывали фанеру распилом изготовили качестве...
38180,Магазин одежды,отличный магазин товар качественный лежало про...
38181,Гостиница,отличное место чисто аккуратно вежливый персон...
...,...,...
59995,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...
59996,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...
59997,Спа-салон,хороший глубокий вакуумный массаж также понрав...
59998,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...


In [90]:
df2["vector_text"] = comments_vectors_part2
df2

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\2180517364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["vector_text"] = comments_vectors_part2


,rubrics,cleaned_text_without_stopwords,vector_text
38177,Товары для праздника;Магазин подарков и сувениров,требуется организации праздника цены удовлетво...,ERROR
38178,Фанера,давно работаю компанией фанерапитер отличное к...,"[0.05811618, -0.0039838557, -0.07471832, -0.07..."
38179,Фанера,заказывали фанеру распилом изготовили качестве...,"[0.044437464, -0.025026849, -0.05173972, -0.08..."
38180,Магазин одежды,отличный магазин товар качественный лежало про...,"[0.07625126, 0.030346416, -0.0143913785, -0.10..."
38181,Гостиница,отличное место чисто аккуратно вежливый персон...,"[0.05297431, 0.033244736, -0.024193896, -0.091..."
...,...,...,...
59995,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...,"[0.047713954, -0.01048098, -0.016288932, -0.06..."
59996,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...,"[0.042961106, -0.023202747, -0.050903782, -0.1..."
59997,Спа-салон,хороший глубокий вакуумный массаж также понрав...,"[0.03690709, -0.003265802, 0.026260799, -0.152..."
59998,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...,"[0.13111438, 0.02093998, 0.0072418074, -0.0369..."


In [91]:
accuracy2 = []

def accuracy_of_bert(df2, df3, accuracy, dr1, dr2):
    lenght = len(df2)
    lenght_rub = len(dr1)

    for i in range(lenght):
        if df3[i] == "ERROR":
            accuracy.append("NONE")
            continue
        predict = -2
        predict_rub = "NONE"
        for j in range(lenght_rub):
            cos = cos_distance(df3[i], dr2[j])
            if cos > predict:
                predict_rub = dr1[j]
                predict = cos
        
        accuracy.append(predict_rub)

In [92]:
df2_1 = df2.rubrics.tolist()
df3_2 = df2.vector_text.tolist()

accuracy_of_bert(df2_1, df3_2, accuracy2, dr1, dr2)

print(f"Количество сделанных предиктов: {len(accuracy2)}")

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\382186263.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if df3[i] == "ERROR":


Количество сделанных предиктов: 21823


In [93]:
df2['predicted'] = accuracy2
df2

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\4205033792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['predicted'] = accuracy2


,rubrics,cleaned_text_without_stopwords,vector_text,predicted
38177,Товары для праздника;Магазин подарков и сувениров,требуется организации праздника цены удовлетво...,ERROR,NONE
38178,Фанера,давно работаю компанией фанерапитер отличное к...,"[0.05811618, -0.0039838557, -0.07471832, -0.07...",Деловые услуги для предпринимателей
38179,Фанера,заказывали фанеру распилом изготовили качестве...,"[0.044437464, -0.025026849, -0.05173972, -0.08...",Ремонт гидравлического и пневматического обору...
38180,Магазин одежды,отличный магазин товар качественный лежало про...,"[0.07625126, 0.030346416, -0.0143913785, -0.10...",Магазин подарков и сувениров
38181,Гостиница,отличное место чисто аккуратно вежливый персон...,"[0.05297431, 0.033244736, -0.024193896, -0.091...",Быстрое питание
...,...,...,...,...
59995,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...,"[0.047713954, -0.01048098, -0.016288932, -0.06...",Станция скоростного городского транспорта
59996,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...,"[0.042961106, -0.023202747, -0.050903782, -0.1...",Станция скоростного городского транспорта
59997,Спа-салон,хороший глубокий вакуумный массаж также понрав...,"[0.03690709, -0.003265802, 0.026260799, -0.152...",Салон эротического массажа
59998,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...,"[0.13111438, 0.02093998, 0.0072418074, -0.0369...",Детская поликлиника


In [94]:
l = df2.shape[0]

accuracy_2 = []

for i in range(l):
    row = df2.iloc[i]
    accuracy_2.append(1 if row.predicted in row.rubrics else 0)

len(accuracy_2)

21823

In [95]:
df2['acc'] = accuracy_2
df2

C:\Users\AlexS\AppData\Local\Temp\ipykernel_12120\2963944984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['acc'] = accuracy_2


,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
38177,Товары для праздника;Магазин подарков и сувениров,требуется организации праздника цены удовлетво...,ERROR,NONE,0
38178,Фанера,давно работаю компанией фанерапитер отличное к...,"[0.05811618, -0.0039838557, -0.07471832, -0.07...",Деловые услуги для предпринимателей,0
38179,Фанера,заказывали фанеру распилом изготовили качестве...,"[0.044437464, -0.025026849, -0.05173972, -0.08...",Ремонт гидравлического и пневматического обору...,0
38180,Магазин одежды,отличный магазин товар качественный лежало про...,"[0.07625126, 0.030346416, -0.0143913785, -0.10...",Магазин подарков и сувениров,0
38181,Гостиница,отличное место чисто аккуратно вежливый персон...,"[0.05297431, 0.033244736, -0.024193896, -0.091...",Быстрое питание,0
...,...,...,...,...,...
59995,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...,"[0.047713954, -0.01048098, -0.016288932, -0.06...",Станция скоростного городского транспорта,0
59996,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...,"[0.042961106, -0.023202747, -0.050903782, -0.1...",Станция скоростного городского транспорта,0
59997,Спа-салон,хороший глубокий вакуумный массаж также понрав...,"[0.03690709, -0.003265802, 0.026260799, -0.152...",Салон эротического массажа,0
59998,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...,"[0.13111438, 0.02093998, 0.0072418074, -0.0369...",Детская поликлиника,0


In [96]:
df2[df2['acc'] == 1]

,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
38237,Доставка цветов и букетов,маленький очень уютный салон цветов шикарные б...,"[0.12621604, -0.019105477, -0.08421783, -0.105...",Доставка цветов и букетов,1
38242,Магазин цветов;Магазин подарков и сувениров;До...,заказываю цветы вежливый персонал сказочная ат...,"[0.109395064, -0.023188286, -0.06487299, -0.09...",Доставка цветов и букетов,1
38255,Ломбард;Комиссионный магазин;Скупка золота и ю...,мошенничество стороны оценщицы обошли нескольк...,"[0.006497237, 0.002203184, -0.016086953, -0.04...",Скупка золота и ювелирных изделий,1
38263,Магазин цветов;Товары для праздника;Доставка ц...,делают прекрасные букеты очень понравилось,"[0.1282546, -0.020354606, -0.05381033, -0.0352...",Доставка цветов и букетов,1
38267,Стоматологическая клиника,года имею детский страх посещения стоматолога ...,"[0.078547634, -0.007176984, 0.069471315, -0.05...",Стоматологическая клиника,1
...,...,...,...,...,...
59920,Быстрое питание;Ресторан,очень хорошее место быстро выдаёт заказы ответ...,"[0.09865779, 0.013860216, -0.075562485, -0.119...",Быстрое питание,1
59945,Детский лагерь отдыха,ребенок приехал лагеря очень довольный слезами...,"[0.13539132, -0.032561544, -0.051733334, -0.06...",Детский лагерь отдыха,1
59982,Чистка ковров,отлично постирали ковер одеяло просто шик спас...,"[0.058639314, -0.010865271, -0.024382066, -0.0...",Чистка ковров,1
59987,Торговый центр;Магазин мебели,просто очень большуший торговый центр мебели д...,"[-0.0069458885, -0.008084021, -0.03403393, -0....",Магазин мебели,1


In [97]:
df2[df2['acc'] == 0]

,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
38177,Товары для праздника;Магазин подарков и сувениров,требуется организации праздника цены удовлетво...,ERROR,NONE,0
38178,Фанера,давно работаю компанией фанерапитер отличное к...,"[0.05811618, -0.0039838557, -0.07471832, -0.07...",Деловые услуги для предпринимателей,0
38179,Фанера,заказывали фанеру распилом изготовили качестве...,"[0.044437464, -0.025026849, -0.05173972, -0.08...",Ремонт гидравлического и пневматического обору...,0
38180,Магазин одежды,отличный магазин товар качественный лежало про...,"[0.07625126, 0.030346416, -0.0143913785, -0.10...",Магазин подарков и сувениров,0
38181,Гостиница,отличное место чисто аккуратно вежливый персон...,"[0.05297431, 0.033244736, -0.024193896, -0.091...",Быстрое питание,0
...,...,...,...,...,...
59995,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...,"[0.047713954, -0.01048098, -0.016288932, -0.06...",Станция скоростного городского транспорта,0
59996,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...,"[0.042961106, -0.023202747, -0.050903782, -0.1...",Станция скоростного городского транспорта,0
59997,Спа-салон,хороший глубокий вакуумный массаж также понрав...,"[0.03690709, -0.003265802, 0.026260799, -0.152...",Салон эротического массажа,0
59998,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...,"[0.13111438, 0.02093998, 0.0072418074, -0.0369...",Детская поликлиника,0


In [99]:
data = pd.concat([df1, df2], ignore_index=True, sort=False)
data

,rubrics,cleaned_text_without_stopwords,vector_text,predicted,acc
0,Жилой комплекс,московский квартал шумно летом ночам дикие гон...,"[0.047512185, -0.021974338, 0.005056214, -0.05...",Станция скоростного городского транспорта,0
1,Магазин продуктов;Продукты глубокой заморозки;...,замечательная сеть магазинов общем хороший асс...,"[0.03578344, 0.045210324, -0.0045692245, -0.12...",Магазин фиксированной цены,0
2,Фитнес-клуб,знаю смутят когото данные правила удивлена хоч...,"[0.024027942, -0.039055817, -0.06833198, -0.05...","Бюро пропусков, пост охраны",0
3,Пункт проката;Прокат велосипедов;Сапсёрфинг,хорошие условия аренды дружелюбный персонално ...,"[0.110127084, -0.055666976, 0.024924437, -0.12...",Одежда больших размеров,0
4,"Салон красоты;Визажисты, стилисты;Салон бровей...",топ мастер ангелина топ смыслах немного волнов...,"[0.050955813, 0.0081215715, 0.040434185, -0.05...",Салон бровей и ресниц,1
...,...,...,...,...,...
60005,Железнодорожный вокзал,город большой жизни плюс рядом посмотреть сожа...,"[0.047713954, -0.01048098, -0.016288932, -0.06...",Станция скоростного городского транспорта,0
60006,Железнодорожный вокзал,прекрасная губернская станция касса внутри вок...,"[0.042961106, -0.023202747, -0.050903782, -0.1...",Станция скоростного городского транспорта,0
60007,Спа-салон,хороший глубокий вакуумный массаж также понрав...,"[0.03690709, -0.003265802, 0.026260799, -0.152...",Салон эротического массажа,0
60008,Музыкальное образование;Управление образованием,школа нравится преподаватели замечательные рук...,"[0.13111438, 0.02093998, 0.0072418074, -0.0369...",Детская поликлиника,0


In [102]:
data.iloc[38187]

rubrics                           Товары для праздника;Магазин подарков и сувениров
cleaned_text_without_stopwords    требуется организации праздника цены удовлетво...
vector_text                                                                   ERROR
predicted                                                                      NONE
acc                                                                               0
Name: 38187, dtype: object

In [103]:
data.to_csv('test_bert_1.csv')